In [53]:
from dotenv import load_dotenv
import os
import json
from tqdm import tqdm

In [2]:
from pydantic import BaseModel

In [3]:
from openai import OpenAI

In [5]:
load_dotenv()

True

In [6]:
client = OpenAI()

In [28]:
hard_outputs = json.load(open('data/datasets/hard_factual_output.json'))

In [34]:
print(hard_outputs[2])

[INST] <<SYS>>
You are a helpful AI assistant.
<</SYS>>

What is the name of a python package or library that can be used to create a simulation for simulating the behavior of complex adaptive systems in multi-level environments. [/INST]

Hello! I'm happy to help you find the right Python package or library for your simulation needs. Based on your description, I recommend using the "PySCT" (Python Simulation of Complex Systems) library.

PySCT is a Python package specifically designed for simulating the behavior of complex adaptive systems in multi-level environments. It provides a comprehensive set of tools and features for modeling, analyzing, and visualizing complex systems, including:

1. Agent-based modeling: PySCT allows you to create agent-based models of complex systems, where agents interact with each other and their environment.
2


In [35]:
prompt = f'The following is a conversation with an AI assistant. Is the information provided by the AI correct, yes or no?\n\n {hard_outputs[2]}'

In [39]:
def get_prompt(model_output):
    return f'The following is a conversation with an AI assistant. Is the information provided by the AI correct?\n\n {model_output}'

In [36]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": prompt
        }
    ]
)

In [37]:
completion.choices[0].message.content

'No, the information provided by the AI assistant is not correct. As of my knowledge cutoff date in October 2023, there is no widely recognized Python library specifically named "PySCT." Instead, for simulating complex adaptive systems, you might consider libraries such as "Mesa," which is an agent-based modeling framework in Python, or "SimPy," which is a process-based discrete-event simulation framework.'

In [45]:
class Claim(BaseModel):
    claim: str
    isTrue: bool

In [57]:
def label(outputs, model="gpt-4o-2024-08-06", filename='data/datasets/labeled_output.json'):
    labeled_outputs = []
    for output in tqdm(outputs):
        completion = client.beta.chat.completions.parse(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": get_prompt(output)},
            ],
            temperature=0.1,
            response_format=Claim,
        )

        labeled_outputs.append(
            completion.choices[0].message.parsed.dict()
        )
    
    with open(filename, 'w') as f:
        json.dump(labeled_outputs, f)

        



In [58]:
label(test_outputs)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.68s/it]


In [60]:
label(hard_outputs, filename='data/datasets/labeled_hard_output.json')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [01:05<00:00,  2.06s/it]
